In [ ]:
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# AIM BOT Detection in CS:GO\n",
    "## COPS Summer of Code 2025 - Intelligence Guild\n",
    "### Computer Vision Week 2 Assignment"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "import torch.nn as nn\n",
    "import torch.optim as optim\n",
    "from torch.utils.data import Dataset, DataLoader, random_split\n",
    "import torchvision.transforms as transforms\n",
    "from torchvision.models.video import r3d_18\n",
    "import cv2\n",
    "import numpy as np\n",
    "import os\n",
    "from sklearn.metrics import classification_report, confusion_matrix\n",
    "import matplotlib.pyplot as plt\n",
    "import time\n",
    "import yaml\n",
    "\n",
    "# Configuration\n",
    "config = {\n",
    "    'data_path': 'data/',\n",
    "    'batch_size': 8,\n",
    "    'epochs': 50,\n",
    "    'lr': 0.001,\n",
    "    'frames_per_clip': 30,\n",
    "    'img_size': (128, 72),\n",
    "    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),\n",
    "    'seed': 42\n",
    "}\n",
    "\n",
    "# Set seed for reproducibility\n",
    "torch.manual_seed(config['seed'])\n",
    "np.random.seed(config['seed'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class CSGODataset(Dataset):\n",
    "    def __init__(self, data_dir, transform=None):\n",
    "        self.data_dir = data_dir\n",
    "        self.transform = transform\n",
    "        self.classes = {'clean': 0, 'aimbot': 1}\n",
    "        self.samples = []\n",
    "        \n",
    "        for label_name, label_val in self.classes.items():\n",
    "            class_dir = os.path.join(data_dir, label_name)\n",
    "            for video_file in os.listdir(class_dir):\n",
    "                if video_file.endswith('.mp4'):\n",
    "                    self.samples.append((os.path.join(class_dir, video_file), label_val))\n",
    "    \n",
    "    def __len__(self):\n",
    "        return len(self.samples)\n",
    "    \n",
    "    def __getitem__(self, idx):\n",
    "        video_path, label = self.samples[idx]\n",
    "        frames = []\n",
    "        \n",
    "        cap = cv2.VideoCapture(video_path)\n",
    "        while cap.isOpened():\n",
    "            ret, frame = cap.read()\n",
    "            if not ret:\n",
    "                break\n",
    "            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n",
    "            frame = cv2.resize(frame, config['img_size'])\n",
    "            frames.append(frame)\n",
    "        cap.release()\n",
    "        \n",
    "        # Sample fixed-length clip\n",
    "        if len(frames) > config['frames_per_clip']:\n",
    "            start_idx = np.random.randint(0, len(frames) - config['frames_per_clip'])\n",
    "            frames = frames[start_idx:start_idx+config['frames_per_clip']]\n",
    "        elif len(frames) < config['frames_per_clip']:\n",
    "            # Pad with last frame\n",
    "            frames += [frames[-1]] * (config['frames_per_clip'] - len(frames))\n",
    "        \n",
    "        # Convert to tensor (T, H, W, C) -> (C, T, H, W)\n",
    "        clip = torch.tensor(np.array(frames)).permute(3, 0, 1, 2).float() / 255.0\n",
    "        \n",
    "        if self.transform:\n",
    "            clip = self.transform(clip)\n",
    "            \n",
    "        return clip, label"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class AIMBotDetector(nn.Module):\n",
    "    def __init__(self):\n",
    "        super().__init__()\n",
    "        # Use pre-trained 3D ResNet\n",
    "        self.backbone = r3d_18(pretrained=True)\n",
    "        \n",
    "        # Replace final layer\n",
    "        num_features = self.backbone.fc.in_features\n",
    "        self.backbone.fc = nn.Sequential(\n",
    "            nn.Linear(num_features, 512),\n",
    "            nn.ReLU(),\n",
    "            nn.Dropout(0.5),\n",
    "            nn.Linear(512, 2)\n",
    "        )\n",
    "        \n",
    "        # Attention mechanism\n",
    "        self.attention = nn.Sequential(\n",
    "            nn.Conv3d(512, 1, kernel_size=1),\n",
    "            nn.Sigmoid()\n",
    "        )\n",
    "    \n",
    "    def forward(self, x):\n",
    "        # Extract features\n",
    "        features = self.backbone.stem(x)\n",
    "        features = self.backbone.layer1(features)\n",
    "        features = self.backbone.layer2(features)\n",
    "        features = self.backbone.layer3(features)\n",
    "        features = self.backbone.layer4(features)\n",
    "        \n",
    "        # Apply attention\n",
    "        attn_weights = self.attention(features)\n",
    "        attn_features = features * attn_weights\n",
    "        \n",
    "        # Pooling\n",
    "        pooled = nn.functional.adaptive_avg_pool3d(attn_features, (1, 1, 1))\n",
    "        pooled = torch.flatten(pooled, 1)\n",
    "        \n",
    "        # Classification\n",
    "        return self.backbone.fc(pooled), attn_weights"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Data augmentation\n",
    "transform = transforms.Compose([\n",
    "    transforms.RandomHorizontalFlip(p=0.5),\n",
    "    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),\n",
    "    transforms.RandomRotation(10)\n",
    "])\n",
    "\n",
    "# Create dataset\n",
    "full_dataset = CSGODataset(config['data_path'], transform=transform)\n",
    "\n",
    "# Split dataset\n",
    "train_size = int(0.8 * len(full_dataset))\n",
    "val_size = len(full_dataset) - train_size\n",
    "train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])\n",
    "\n",
    "# Create data loaders\n",
    "train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)\n",
    "val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize model\n",
    "model = AIMBotDetector().to(config['device'])\n",
    "\n",
    "# Loss and optimizer\n",
    "criterion = nn.CrossEntropyLoss()\n",
    "optimizer = optim.Adam(model.parameters(), lr=config['lr'])\n",
    "\n",
    "# Training loop\n",
    "train_losses, val_losses = [], []\n",
    "train_accs, val_accs = [], []\n",
    "\n",
    "for epoch in range(config['epochs']):\n",
    "    # Training\n",
    "    model.train()\n",
    "    running_loss = 0.0\n",
    "    correct = 0\n",
    "    total = 0\n",
    "    start_time = time.time()\n",
    "    \n",
    "    for inputs, labels in train_loader:\n",
    "        inputs, labels = inputs.to(config['device']), labels.to(config['device'])\n",
    "        \n",
    "        optimizer.zero_grad()\n",
    "        outputs, _ = model(inputs)\n",
    "        loss = criterion(outputs, labels)\n",
    "        loss.backward()\n",
    "        optimizer.step()\n",
    "        \n",
    "        running_loss += loss.item()\n",
    "        _, predicted = outputs.max(1)\n",
    "        total += labels.size(0)\n",
    "        correct += predicted.eq(labels).sum().item()\n",
    "    \n",
    "    train_loss = running_loss / len(train_loader)\n",
    "    train_acc = 100 * correct / total\n",
    "    train_losses.append(train_loss)\n",
    "    train_accs.append(train_acc)\n",
    "    \n",
    "    # Validation\n",
    "    model.eval()\n",
    "    val_loss = 0.0\n",
    "    correct = 0\n",
    "    total = 0\n",
    "    all_preds = []\n",
    "    all_labels = []\n",
    "    \n",
    "    with torch.no_grad():\n",
    "        for inputs, labels in val_loader:\n",
    "            inputs, labels = inputs.to(config['device']), labels.to(config['device'])\n",
    "            outputs, _ = model(inputs)\n",
    "            loss = criterion(outputs, labels)\n",
    "            \n",
    "            val_loss += loss.item()\n",
    "            _, predicted = outputs.max(1)\n",
    "            total += labels.size(0)\n",
    "            correct += predicted.eq(labels).sum().item()\n",
    "            \n",
    "            all_preds.extend(predicted.cpu().numpy())\n",
    "            all_labels.extend(labels.cpu().numpy())\n",
    "    \n",
    "    val_loss = val_loss / len(val_loader)\n",
    "    val_acc = 100 * correct / total\n",
    "    val_losses.append(val_loss)\n",
    "    val_accs.append(val_acc)\n",
    "    \n",
    "    epoch_time = time.time() - start_time\n",
    "    \n",
    "    print(f\"Epoch {epoch+1}/{config['epochs']} | \"\n",
    "          f\"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | \"\n",
    "          f\"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | \"\n",
    "          f\"Time: {epoch_time:.2f}s\")\n",
    "\n",
    "# Save model\n",
    "torch.save(model.state_dict(), 'aimbot_detector.pth')\n",
    "\n",
    "# Plot results\n",
    "plt.figure(figsize=(12, 5))\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.plot(train_losses, label='Train Loss')\n",
    "plt.plot(val_losses, label='Val Loss')\n",
    "plt.legend()\n",
    "plt.title('Loss Curve')\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.plot(train_accs, label='Train Accuracy')\n",
    "plt.plot(val_accs, label='Val Accuracy')\n",
    "plt.legend()\n",
    "plt.title('Accuracy Curve')\n",
    "plt.savefig('training_results.png')\n",
    "\n",
    "# Classification report\n",
    "print(\"\\nClassification Report:\")\n",
    "print(classification_report(all_labels, all_preds, target_names=['Clean', 'Aimbot']))\n",
    "\n",
    "# Confusion matrix\n",
    "cm = confusion_matrix(all_labels, all_preds)\n",
    "plt.figure(figsize=(8, 6))\n",
    "plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)\n",
    "plt.title('Confusion Matrix')\n",
    "plt.colorbar()\n",
    "tick_marks = np.arange(2)\n",
    "plt.xticks(tick_marks, ['Clean', 'Aimbot'])\n",
    "plt.yticks(tick_marks, ['Clean', 'Aimbot'])\n",
    "plt.ylabel('True Label')\n",
    "plt.xlabel('Predicted Label')\n",
    "plt.savefig('confusion_matrix.png')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualization function\n",
    "def visualize_attention(video_path):\n",
    "    cap = cv2.VideoCapture(video_path)\n",
    "    frames = []\n",
    "    \n",
    "    while cap.isOpened():\n",
    "        ret, frame = cap.read()\n",
    "        if not ret:\n",
    "            break\n",
    "        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n",
    "        frame = cv2.resize(frame, config['img_size'])\n",
    "        frames.append(frame)\n",
    "    cap.release()\n",
    "    \n",
    "    # Process clip\n",
    "    clip = torch.tensor(np.array(frames[:config['frames_per_clip']])).permute(3, 0, 1, 2).float() / 255.0\n",
    "    clip = clip.unsqueeze(0).to(config['device'])\n",
    "    \n",
    "    # Get predictions and attention\n",
    "    model.eval()\n",
    "    with torch.no_grad():\n",
    "        pred, attn_weights = model(clip)\n",
    "    \n",
    "    # Process attention\n",
    "    attn_weights = attn_weights.squeeze().cpu().numpy()\n",
    "    \n",
    "    # Create overlay\n",
    "    for i in range(len(frames)):\n",
    "        frame = frames[i]\n",
    "        heatmap = cv2.resize(attn_weights[i], (frame.shape[1], frame.shape[0]))\n",
    "        heatmap = (heatmap * 255).astype(np.uint8)\n",
    "        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)\n",
    "        overlay = cv2.addWeighted(frame, 0.7, heatmap, 0.3, 0)\n",
    "        \n",
    "        plt.figure(figsize=(10, 6))\n",
    "        plt.subplot(1, 2, 1)\n",
    "        plt.imshow(frame)\n",
    "        plt.title('Original Frame')\n",
    "        plt.axis('off')\n",
    "        \n",
    "        plt.subplot(1, 2, 2)\n",
    "        plt.imshow(overlay)\n",
    "        plt.title('Attention Map')\n",
    "        plt.axis('off')\n",
    "        plt.tight_layout()\n",
    "        plt.show()\n",
    "\n",
    "# Example usage\n",
    "# visualize_attention('data/aimbot/sample.mp4')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Key Features of the Implementation\n",
    "1. **3D CNN Architecture**: Uses PyTorch's pre-trained R3D-18 model for spatiotemporal feature extraction\n",
    "2. **Attention Mechanism**: Visualizes regions of interest in gameplay frames\n",
    "3. **Data Augmentation**: Horizontal flips, color jitter, and rotation for robustness\n",
    "4. **Visualization Tools**: Attention mapping and performance metrics\n",
    "5. **Efficient Processing**: Frame sampling and GPU acceleration\n",
    "\n",
    "## Usage Instructions\n",
    "1. Create dataset folder structure:\n",
    "   ```\n",
    "   data/\n",
    "   ├── aimbot/\n",
    "   │   ├── video1.mp4\n",
    "   │   └── ...\n",
    "   └── clean/\n",
    "       ├── video1.mp4\n",
    "       └── ...\n",
    "   ```\n",
    "2. Run all notebook cells\n",
    "3. Visualize results with `visualize_attention()` function"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4